# dbatools Morning Checklist

Here's a sample PowerShell notebook showing how you can use dbatools to do a quick morning check-in of your SQL Server estate.

## First let's get setup
- Import the dbatools module
- Store our list of servers in a variable
  - I'm using `Get-DbaRegisteredServer` to use my registered server list, but you could also manually specify server names or use `Import-Csv` to get them from a CSV file.


In [1]:
Import-Module dbatools

$servers = Get-DbaRegisteredServer -Group Sqlserver

$servers


Name   ServerName Group     Description Source             
----   ---------- -----     ----------- ------             
mssql1 mssql1     SqlServer             Local Server Groups
mssql2 mssql2     SqlServer             Local Server Groups




## 1. Check SQL Server Backups

First we'll use dbatools to check on all our database backups.

In [2]:
Get-DbaLastBackup -SqlInstance $servers | 
Format-Table SqlInstance, Database, LastFullBackup, LastDiffBackup, LastLogBackup -AutoSize


SqlInstance Database           LastFullBackup          LastDiffBackup          LastLogBackup          
----------- --------           --------------          --------------          -------------          
mssql1      AdventureWorks2017 2019-12-18 05:46:10.000                                                
mssql1      DatabaseAdmin      2019-12-18 05:46:25.000                                                
mssql1      master                                                                                    
mssql1      model                                                                                     
mssql1      msdb                                                                                      
mssql2      AdventureWorks2019                                                                        
mssql2      master             2019-12-18 05:46:19.000                                                
mssql2      model              2019-12-18 05:46:20.000 2019-12-18 05:46:

You could also use the PowerShell `Where-Object` to filter out the databases that meet your backup requirements. 

In this instance:
 - Full backup within the last 7 days
 - Differential backup within the last 1 day

In [3]:
Get-DbaLastBackup -SqlInstance $servers | 
Where-Object {($_.LastFullBackup.Date -lt ((Get-Date).AddDays(-7))) `
-or ($_.LastDiffBackup.Date -lt ((Get-Date).AddDays(-1)))} |
Format-Table SqlInstance, Database, LastFullBackup, LastDiffBackup, RecoveryModel -AutoSize


SqlInstance Database           LastFullBackup          LastDiffBackup RecoveryModel
----------- --------           --------------          -------------- -------------
mssql1      AdventureWorks2017 2019-12-18 05:46:10.000                       Simple
mssql1      DatabaseAdmin      2019-12-18 05:46:25.000                         Full
mssql1      master                                                           Simple
mssql1      model                                                              Full
mssql1      msdb                                                             Simple
mssql2      AdventureWorks2019                                               Simple
mssql2      master             2019-12-18 05:46:19.000                       Simple




## 2. Check Last Good DBCC Checks

For step two we'll make sure we're running DBCC CheckDb regularly against our databases.

In [2]:
(Get-DbaLastGoodCheckDb -SqlInstance $servers -ExcludeDatabase tempdb).where({$_.Status -ne 'OK'}) |
Format-Table SqlInstance, Database, DaysSinceLastGoodCheckDb, Status, DataPurityEnabled -Autosize


SqlInstance Database           DaysSinceLastGoodCheckDb Status                        DataPurityEnabled
----------- --------           ------------------------ ------                        -----------------
mssql1      master                                      CheckDB should be performed                True
mssql1      model                                       CheckDB should be performed                True
mssql2      master                                      CheckDB should be performed                True
mssql2      model                                       CheckDB should be performed                True
mssql2      msdb                                        CheckDB should be performed                True
mssql2      AdventureWorks2019                          New database, not checked yet              True




## 3. Check for Failed Jobs

We can also check across our entire SQL Server estate for any failed jobs.

In [3]:
Get-DbaAgentJob -SqlInstance $servers -ExcludeDisabledJobs | 
Where-Object LastRunOutcome -ne 'Succeeded' | 
Format-Table SqlInstance, Name, LastRunDate, LastRunOutcome


SqlInstance Name            LastRunDate          LastRunOutcome
----------- ----            -----------          --------------
mssql1      ImportantETLJob 1/1/0001 12:00:00 AM        Unknown
mssql2      DailyReport     3/30/2020 2:02:40 PM         Failed




## 4. Check SQL Server Error Log Messages

There is also a lot of useful information in the SQL Server Error Log that dbatools can help you look into. We can look at all error log entries.

In [6]:
Get-DbaErrorLog -SqlInstance $servers -After (get-date).AddDays(-1) |
Select-object -First 10 | # I'm only showing the first 10 for sake of space in this demo, remove this for real use cases. 
Format-Table LogDate, SqlInstance, SqlInstance, Source, Text 


LogDate               SqlInstance SqlInstance Source Text                                                              
-------               ----------- ----------- ------ ----                                                              
12/18/2019 5:45:28 AM mssql1      mssql1      Server Microsoft SQL Server 2017 (RTM-CU11) (KB4462262) - 14.0.3038.14...
12/18/2019 5:45:28 AM mssql1      mssql1      Server UTC adjustment: -5:00                                             
12/18/2019 5:45:28 AM mssql1      mssql1      Server (c) Microsoft Corporation.                                        
12/18/2019 5:45:28 AM mssql1      mssql1      Server All rights reserved.                                              
12/18/2019 5:45:28 AM mssql1      mssql1      Server Server process ID is 4120.                                        
12/18/2019 5:45:28 AM mssql1      mssql1      Server Logging SQL Server messages in file '/var/opt/mssql/log/errorlog'.
12/18/2019 5:45:28 AM mssql1      mssql

We can also filter to look for specific issues, like login failures.

In [7]:
Get-DbaErrorLog -SqlInstance $servers -After (get-date).AddDays(-1) -Text "Login Failed" |
Format-Table LogDate, SqlInstance, SqlInstance, Source, Text


LogDate               SqlInstance SqlInstance Source Text                                                              
-------               ----------- ----------- ------ ----                                                              
12/18/2019 6:00:59 AM mssql1      mssql1      Logon  Login failed for user 'sa'. Reason: Password did not match that...
12/18/2019 6:01:13 AM mssql1      mssql1      Logon  Login failed for user 'BadLogin'. Reason: Could not find a logi...




## 5. Add your custom checks

Perhaps in your environment you have additional things to check.

- **Get-DbaAvailabilityGroup** - Check your availability groups and determine which node is primary 
- **Get-DbaDiskSpace** - Check the available disk space on your servers and ensure you're not close to running out
- **Test-DbaDbOwner** - Make sure all your databases are owned by sa, or specify an owner with the `-TargetLogin` Parameter
- **Test-DbaDbRecoveryModel** - Check your database recovery models are as expected, including not in pseudo simple mode
- **Test-DbaRepLatency** - Test replication latency with tracer tokens
